In [ ]:

Анализ поведения пользователей в мобильном приложении

Цель исследования: выделить основные сценарии использования приложения и предложить гипотезы по улучшению юзер экспириенса

Предобработка данных:

загрузка и чтение датасетов;
проверка типов данных;
приведение названий столбцов к snake_case;
проверка данных на наличие пропусков и дубликатов.
Исследовательский анализ данных:
изучение количества событий каждого типа;
анализ распределения количества действий юзерами;
анализ среднего количества действий на юзера по каналу привлечения;
расчет метрик - DAU, WAU, retention rate на N-ый день - по ходу исследования набор метрик может корректироваться;
изучение различий в паттернах поведения юзеров, использующих и игнорирующих рекомендации:
анализ доли целевого трафика по каналу привлечения (целевой трафик - юзеры, использующие рекомендации);
изучение количества просмотренных контактов на 1 юзера для каждого сегмента;
анализ конверсии в целевое действие - просмотр контактов - по этим сегментам;
Проведение стат. тестов:
Конверсия в просмотры контактов у пользователей, совершающих действия tips_show и tips_click, отличается от тех, кто только смотрит рекомендации - только tips_show
H0: конверсия в просмотр контактов у двух сегментов юзеров одинаковая;
H1: конверсия в просмотр контактов у двух сегментов юзеров разная;
выводы по полученной информации;
2)Конверсия в просмотр контактов из поиска и рекомендаций одинаковая

то есть мы объединяем все серчи в один блок - подразумевается, что это какие-либо фильтры, поиски по популярным названия / категориям товаров или кастомный поиск - получаем в результате теста метрику "качества" наших рекомендаций;
таким образом, с помощью этой гипотезы мы сможем понять насколько точны наши рекомендации, то есть если конверсия в просмотр контактов по рекомендованным объявлениям гораздо ниже, то стоит сообщить об этом команде, занимающейся алгоритмами рекомендаций
H0: конверсия в просмотр контактов из поиска и рекомендаций одинаковая;
H1: конверсия в просмотр контактов из поиска и рекомендаций разная;
выводы по полученной информации;
Проработка основных вопросов анализа:
Какие сценарии использования приложения выделяются?
выделение наиболее популярных сценариев;
предположения того, почему сложилась именно такие сценарии использования;
выводы по полученной информации;
Как различается время между событиями advert_open -> contacts_show и tips_click -> contacts_show. Какая конверсия в целевое действие у данных действий?
расчет необходимых метрик;
выводы по полученной информации;
Выводы и рекомендации
Для более быстрого погружения в ситуацию предоставляю описание данных:

Датасет содержит данные о событиях, совершенных в мобильном приложении "Ненужные вещи". В нем пользователи продают свои ненужные вещи, размещая их на доске объявлений.

В датасете содержатся данные пользователей, впервые совершивших действия в приложении после 7 октября 2019 года.

Колонки в mobile_sources.csv:

userId — идентификатор пользователя,
source — источник, с которого пользователь установил приложение.
Колонки в mobile_dataset.csv:

event.time — время совершения,
user.id — идентификатор пользователя,
event.name — действие пользователя.
Виды действий:

advert_open — открыл карточки объявления,
photos_show — просмотрел фотографий в объявлении,
tips_show — увидел рекомендованные объявления,
tips_click — кликнул по рекомендованному объявлению,
contacts_show и show_contacts — посмотрел номер телефона,
contacts_call — позвонил по номеру из объявления,
map — открыл карту объявлений,
search_1—search_7 — разные действия, связанные с поиском по сайту,
favorites_add — добавил объявление в избранное.
Шаг 2. Предобработка данных
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import statistics as stat
from scipy import stats as st
import math as mth
from plotly.subplots import make_subplots
from tqdm.auto import tqdm
import warnings

# импортируем библиотеки
# скроем будущие предупреждения об ошибках
warnings.filterwarnings('ignore')
mob_df = pd.read_csv('https://code.s3.yandex.net/datasets/mobile_dataset.csv')
mob_sources = pd.read_csv('https://code.s3.yandex.net/datasets/mobile_sources.csv')
# загружаем датасеты
mob_df
# смотрим на датасет с ифнормацией о действиях юзеров
event.time	event.name	user.id
0	2019-10-07 00:00:00.431357	advert_open	020292ab-89bc-4156-9acf-68bc2783f894
1	2019-10-07 00:00:01.236320	tips_show	020292ab-89bc-4156-9acf-68bc2783f894
2	2019-10-07 00:00:02.245341	tips_show	cf7eda61-9349-469f-ac27-e5b6f5ec475c
3	2019-10-07 00:00:07.039334	tips_show	020292ab-89bc-4156-9acf-68bc2783f894
4	2019-10-07 00:00:56.319813	advert_open	cf7eda61-9349-469f-ac27-e5b6f5ec475c
...	...	...	...
74192	2019-11-03 23:53:29.534986	tips_show	28fccdf4-7b9e-42f5-bc73-439a265f20e9
74193	2019-11-03 23:54:00.407086	tips_show	28fccdf4-7b9e-42f5-bc73-439a265f20e9
74194	2019-11-03 23:56:57.041825	search_1	20850c8f-4135-4059-b13b-198d3ac59902
74195	2019-11-03 23:57:06.232189	tips_show	28fccdf4-7b9e-42f5-bc73-439a265f20e9
74196	2019-11-03 23:58:12.532487	tips_show	28fccdf4-7b9e-42f5-bc73-439a265f20e9
74197 rows × 3 columns

Необходимо привести названия столбцов к snake_case, а далее посмотрим общую информацию по датасету

mob_df.columns = mob_df.columns.str.replace('.', '_')
# приводим столбцы к snake_case
mob_df.info()
# смотрим общую инфу по дф
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74197 entries, 0 to 74196
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   event_time  74197 non-null  object
 1   event_name  74197 non-null  object
 2   user_id     74197 non-null  object
dtypes: object(3)
memory usage: 1.7+ MB
Тип данных у столбца с датой события неверный, необходимо его привести к datetime, а также поскольку мы имеем много знаков после запятой - округлим время до секунд, однако перед началом работы со столбцом 'event_time' стоит проверить, есть ли у нас дубликаты. Так как после округления времени, они могут появиться - но если до округления их не было, то стоит оставить данные дубли.

mob_df.duplicated().sum()
# количество явных дубликатов
0
mob_df['event_time'] = pd.to_datetime(mob_df['event_time'], format='%Y.%m.%d %H:%M:%S').dt.round('1S')
# приводим столбец с о временем события в порядок
mob_df.duplicated().sum()
# количество явных дубликатов после округления времени события
1118
Итоги работы с датасетом с ифнормацией о действиях юзеров:

пропусков и явных дубликатов нет;
названия столбцов приведены к snake_case;
тип данных столбца 'event_time' приведен к datetime, а также время события округлено до секунд;
несмотря на то, что у нас появилось 1118 дубликатов, мы их не будем удалять, так как это произошло из-за округления времени события до секунд
mob_sources
# смотрим на датасет с ифнормацией об источниках трафика
userId	source
0	020292ab-89bc-4156-9acf-68bc2783f894	other
1	cf7eda61-9349-469f-ac27-e5b6f5ec475c	yandex
2	8c356c42-3ba9-4cb6-80b8-3f868d0192c3	yandex
3	d9b06b47-0f36-419b-bbb0-3533e582a6cb	other
4	f32e1e2a-3027-4693-b793-b7b3ff274439	google
...	...	...
4288	b86fe56e-f2de-4f8a-b192-cd89a37ecd41	yandex
4289	424c0ae1-3ea3-4f1e-a814-6bac73e48ab1	yandex
4290	437a4cd4-9ba9-457f-8614-d142bc48fbeb	yandex
4291	c10055f0-0b47-477a-869e-d391b31fdf8f	yandex
4292	d157bffc-264d-4464-8220-1cc0c42f43a9	google
4293 rows × 2 columns

mob_sources = mob_sources.rename(columns={'userId': 'user_id'})
# приводим столбцы к snake_case
mob_sources.info()
# смотрим общую инфу по дф
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4293 entries, 0 to 4292
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  4293 non-null   object
 1   source   4293 non-null   object
dtypes: object(2)
memory usage: 67.2+ KB
mob_sources.duplicated().sum()
# количество явных дубликатов
0
Пропусков и дубликатов также нет, названия столбцов приведены к snake_case

Итоги предобработки анализ данных:

пропусков и явных дубликатов в первоначаьных данных нет;
названия столбцов приведены к snake_case;
для датасета mob_df тип данных столбца 'event_time' приведен к datetime, а также время события округлено до секунд;
несмотря на то, что у нас появилось 1118 дубликатов, мы их не будем удалять, так как это произошло из-за округления времени события до секунд
Шаг 3. Исследовательский анализ данных
# объединим два дф в один для более удобной работы
mob_df = mob_df.merge(mob_sources, how='left', on='user_id')
# посмотрим сколько событий каждого типа содержится в нашем дф
mob_df['event_name'].value_counts()
tips_show        40055
photos_show      10012
advert_open       6164
contacts_show     4450
map               3881
search_1          3506
favorites_add     1417
search_5          1049
tips_click         814
search_4           701
contacts_call      541
search_3           522
search_6           460
search_2           324
search_7           222
show_contacts       79
Name: event_name, dtype: int64
Поскольку мы не имеем точной информации о том, какое именно действие подразумевается под "search_1" - "search_7", то мы объединим их все в одну категорию "search".

Также мы имеем 2 события очень похожих друг на друга - "contacts_show" и "show_contacts" - скорее всего, данные хранятся в двух БД, поэтому при выгрузке получилась подобная ситуация. Данные события мы объединим в категорию "contacts_show"

# заменим значения в соответствии с планом выше
mob_df['event_name'] = mob_df['event_name'].str.replace('search_+\d', 'search', regex=True)
mob_df.loc[mob_df['event_name'] == 'show_contacts', 'event_name'] = 'contacts_show'
# проверим корректность наших преобразований
mob_df['event_name'].value_counts()
tips_show        40055
photos_show      10012
search            6784
advert_open       6164
contacts_show     4529
map               3881
favorites_add     1417
tips_click         814
contacts_call      541
Name: event_name, dtype: int64
# построим гистограмму распределения количества действий каждого пользователя
mob_df.groupby('user_id')['event_time'].count().hist()

plt.title('Гистограмма распределения количества действий каждого пользователя', loc='left')
plt.xlabel('Количество действий на юзера')
plt.ylabel('Количество юзеров')

plt.show()

По гистограмме видно, что большинство пользователей совершает до 50 действий, а также в данных есть выбросы - юзеры, совершившие порядка 500 событий. Выбросы пока что мы оставим, возможно, удастся понять, почему они совершают столько действий.

Далее мы изучим, сколько юзеров переходят по нашим рекомендациям, а сколько игнорируют их. Возможно, дело как раз таки в рекомендациях, и юзеры, использующих их, совершают в среднем гораздо больше действий.

Далее мы посмотрим основную стат. информацию по количеству событий на юзера

# выводим основную стат. информацию по столбцу с количеством действий каждого юзера
mob_df.groupby('user_id')['event_time'].count().describe()
count    4293.000000
mean       17.283252
std        29.130677
min         1.000000
25%         5.000000
50%         9.000000
75%        17.000000
max       478.000000
Name: event_time, dtype: float64
В среднем наши юзеры совершают по 17 действий, медианное же значение - 9 действий. Из этого можно сделать следующие выводы:

у нас есть большой "хвост" из юзеров, которые соверашают очень мало действий, но также 25% крайне активных юзеров - с количеством событий сильно больше среднего значения - 17 - которые и перекашивают наше распределение (25% потому что у нас есть информация о 3 квартиле (Q3), который равен 17 событиям);
распределение имеет положительную ассиметрию - правосторонний скос;
так же по максимальному значению - 478 действий - можно понять, что среди наших 25% активных пользователей есть сегмент пользователей, который даже на фоне этих 25% самых активных юзеров, будет сильно выделяться - скорее всего, это люди, занимающиеся перепродажей или магазины / мелкие предприниматели, которые на нашей площадке продают не только свои ненужные вещи;
учитывя поинты выше, и факт того, что стандартное отклонение равно 29, при среднем 17 - это подтверждает, что у нас есть много пользователей с небольшой активностью 5 - 9 событий, а также немало юзеров с крайне высокой активностью - сильно больше 17;
Далее мы перейдем к изучению поведения пользователей, которые используют наши рекомендации, и которые игнорируют их.

# создадим дф с пользователями, использующими рекомендации
tips_lovers =  mob_df[mob_df['user_id'].isin(mob_df[mob_df['event_name'] == 'tips_click']['user_id'])]
# выведем кол-во таких юзеров
tips_lovers['user_id'].nunique()
322
# создадим дф с пользователями, не использующими рекомендации
tips_haters = mob_df[~mob_df['user_id'].isin(tips_lovers['user_id'])]
# выведем кол-во таких юзеров
tips_haters['user_id'].nunique()
3971
Таким образом, в нашем датасете содержится информация о 322 пользователях, которые переходят по нашим рекомендациям, и 3971 - не переходят.

Анализ различий в поведении пользователей, использующих и игнорирующих рекомендации
Далее мы узнаем, сколько в среднем просмотров контактов приходится на пользователя, и одинаково ли это значение для двух исследуемых категорий пользователей - использующих и игнорирующих рекомендации (реки)

# посчитаем общее количество просмотренных контактов юзерами, использующими реки
(tips_lovers[tips_lovers['event_name'].isin(['contacts_show'])]
          .groupby('user_id', as_index=False)['event_name']
          .count()['event_name'].sum())
869
# посчитаем количество просмотренных контактов на 1 юзера, использующих реки
(tips_lovers[tips_lovers['event_name'].isin(['contacts_show'])]
          .groupby('user_id', as_index=False)['event_name']
          .count()['event_name'].sum() / tips_lovers['user_id'].nunique())
2.698757763975155
Таким образом, 322 пользователя, которые переходят по нашим рекомендациям, просмотрели 869 контактов продавцов или же почти 2,7 просмотра контакта на юзера

# посчитаем общее количество просмотренных контактов юзерами, не использующих реки
(tips_haters[tips_haters['event_name'].isin(['contacts_show'])]
          .groupby('user_id', as_index=False)['event_name']
          .count()['event_name'].sum())
3660
# посчитаем количество просмотренных контактов на 1 юзера, не использующего реки
(tips_haters[tips_haters['event_name'].isin(['contacts_show'])]
          .groupby('user_id', as_index=False)['event_name']
          .count()['event_name'].sum() / tips_haters['user_id'].nunique())
0.9216821959204231
В случае с 3971 пользователями, которые игнорят по наши рекомендации, просмотров контактов продавцов оказолось лишь 3660 или же 0,92 просмотра контакта на юзера

Далее мы проверим на статистическом тесте гипотезу о равнестве конверсии в просмтор контакта между 2 сегментами пользователей - использующих и игнорирующих наши рекомендации - и выясним совпадает ли конверсия в просмотр контактов

# осортируем данные по айдишнику юзера и дате ивента
# чтобы корректно разбить действия по сессиям
mob_df = mob_df.sort_values(['user_id', 'event_time'])
# зададим начальное значение и шаг для выбора интервала
initial_interval = pd.Timedelta('5 minutes')
increment = pd.Timedelta('10 minutes')

results = []

for i in range(4):
    # определяем интервал
    interval = initial_interval + i * increment

    # считаем количество сессий при заданном интервале
    g = (mob_df.groupby('user_id')['event_time'].diff() > interval).cumsum()
    mob_df['session_id'] = mob_df.groupby(['user_id', g], sort=False).ngroup() + 1
    num_sessions = mob_df['session_id'].nunique()

    # добавляем полученной значение в список
    results.append([str(interval), num_sessions])

# создаем дф из полученных данных
df = pd.DataFrame(results, columns=['interval_time', 'number_of_sessions'])
df['interval_time'] = df['interval_time'].apply(lambda x: str(x).split()[-1])
fig = go.Figure()

# строим график
fig.add_trace(go.Scatter(
    x=df['interval_time'],
    y=df['number_of_sessions'],
    mode='lines+markers',
    name='количество сессий'
))

# Update the layout
fig.update_layout(
    title='Зависимость количества сессий от выбранного значения таймаута',
    xaxis_title='значение таймаута',
    yaxis_title='количество сессий'
)

# Show the plot
fig.show()
Таким образом, у нас получилось 10633 сессии.

Интервал в 25 минут был выбран на основании того, что с интервалом в 15 минут у нас было бы на 1000 сессий больше, а вот далее, после 25 минутного интервала, количество сессий уменьшалось более медленными темпами - по 500 и менее, а значит наиболее оптимальное время сессии - 25 минут.

# возьмем интервал в 25 минут
g = (mob_df.groupby('user_id')['event_time'].diff() > pd.Timedelta('25Min')).cumsum()
# добавляем столбец с номером сессии
mob_df['session_id'] = mob_df.groupby(['user_id', g], sort=False).ngroup() + 1
# проверяем получившийся результат
mob_df
event_time	event_name	user_id	source	session_id
805	2019-10-07 13:39:46	tips_show	0001b1d5-b74a-4cbf-aeb0-7df5947bf349	other	1
806	2019-10-07 13:40:31	tips_show	0001b1d5-b74a-4cbf-aeb0-7df5947bf349	other	1
809	2019-10-07 13:41:06	tips_show	0001b1d5-b74a-4cbf-aeb0-7df5947bf349	other	1
820	2019-10-07 13:43:21	tips_show	0001b1d5-b74a-4cbf-aeb0-7df5947bf349	other	1
830	2019-10-07 13:45:31	tips_show	0001b1d5-b74a-4cbf-aeb0-7df5947bf349	other	1
...	...	...	...	...	...
72584	2019-11-03 15:51:24	tips_show	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10633
72589	2019-11-03 15:51:58	contacts_show	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10633
72684	2019-11-03 16:07:41	tips_show	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10633
72688	2019-11-03 16:08:18	tips_show	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10633
72689	2019-11-03 16:08:25	tips_show	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10633
74197 rows × 5 columns

Удалить комменты ревьювера ниже этого поинта
Проверка гипотезы: конверсия по сессиям в просмотры контактов различается у пользователей, просматривающих и игнорирующих рекомендации
Сначала мы проведем анализ по сессиям, а далее перейдем к сравнению конверсии по двум сегментам - пользователей, просматривающих и игнорирующих рекомендации - это позволит нам выявить, разницу между конверсией по сессиям и по юзерам, что даст нам более широкое представление о поведении юзеров в нашем приложении

Расчет конверсии по сессимя

# выделяем сессии юзеров, использующих реки
tips_session = mob_df.query('event_name == "tips_click"')['session_id'].unique()

# выделяем сессии юзеров, не использующих реки
no_tips_session = mob_df.query('session_id not in @tips_session')['session_id'].unique()

# выделяем сессии, в которых юзеры посмотрели контакты
contacts_session = mob_df.query('event_name == "contacts_show"')['session_id'].unique()

# выделяем сессии, в которых юзеры кликнули по рекам и посмотрели контакты
tips_contacts_session = set(tips_session) & set(contacts_session)

# выделяем сессии, в которых юзеры не кликали по рекам, но посмотрели контакты
no_tips_contacts_session = set(no_tips_session) & set(contacts_session)
tst = mob_df.copy()
tst.sort_values(['session_id', 'event_time'], inplace=True)

# удалим повторы в сессиях
tst['session_id'] = tst['session_id'].drop_duplicates()
tst = tst.dropna()

# вернем столбец 'session_id' к типу инт
tst['session_id'] = tst['session_id'].astype('int')

# добавим столбец 'is_tips' для выделения юзеров, использующих реки
tst['is_tips'] =  tst['session_id'].apply(lambda x: True if x in tips_session else False)

# добавим столбец 'is_success' для выделения сессий, в которых юзеры просмотрели контакты
tst['is_success'] = tst.apply(
    lambda row: True if (row['session_id'] in tips_contacts_session)
    or (row['session_id'] in no_tips_contacts_session)
    else False,
    axis=1
)
# считаем конверсию по сессиям в целевое действие у юзеров, использующих реки
tst[tst['is_tips'] == True]['is_success'].mean()
0.21770334928229665
# считаем конверсию по сессиям в целевое действие у юзеров, не использующих реки
tst[tst['is_tips'] == False]['is_success'].mean()
0.16250611845325502
Как можно заметить, конверсия в просмотр контактов разная, однако без проведения стат. теста мы не можем с уверенностью утверждать, что мы имеем стат. значимое различие, так что переходим к проведению стат. теста

H0: конверсия в просмотр контактов для двух сегментов одинаковая

H1: конверсия значение просмотр контактов для двух сегментов разная

# задаем уровень стат. значимости
alpha = .05

# проводим стат. тест о равенстве средних
results = st.ttest_ind(tst[tst['is_tips'] == True]['is_success'],
                       tst[tst['is_tips'] == False]['is_success'], equal_var=False)

# выводим получившееся значение p-value и результаты стат. теста
print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')
p-значение: 0.007462528572232172
Отвергаем нулевую гипотезу
P-value < 0.05, а значит мы отвергаем нулевую гипотезу и приримаем альтернативную (H1) - "конверсия в просмотр контактов по сессиям для двух сегментов разная"

Так оно и есть - юзеры, просматривающие наши рекомендации, в среднем активнее более чем в 2 раза - 39 действий против 16.

С точки зрения нашей платформы этот инсайт можно трактовать по-разному - с одной стороны, юзеры определенно точно проводят больше времени в приложении, с другой - конверсия в просмотр объявления ниже, а значит наши рекомендации не совсем точны. Или же дело в паттернах поведения?

Возможно, юзеры, которые просмтривают наши рекомендации, имеют больше свободного времени, поэтому для них наши рекомендации, являются генератором контента для просмотра.

Конверсию в целевое действие - просмотр контактов - по юзерам мы рассмотрим в блоке "Проработки основных вопросов исследования"

# создадим дф с количеством действий каждого типа
# а также посчитаем долю в общем количестве логов для юзеров, использующих рекомендации
lovers_evnt = tips_lovers['event_name'].value_counts() \
                         .to_frame() \
                         .reset_index() \
                         .rename(columns={'index': 'event_name', 'event_name': 'am_actions'})
lovers_evnt['share'] = lovers_evnt['am_actions'] / lovers_evnt['am_actions'].sum()
lovers_evnt = lovers_evnt.sort_values(by='share').reset_index()
# создадим дф с количеством действий каждого типа
# а также посчитаем долю в общем количестве логов для юзеров, игнорирующих рекомендации
haters_evnt = tips_haters['event_name'].value_counts() \
                         .to_frame() \
                         .reset_index() \
                         .rename(columns={'index': 'event_name', 'event_name': 'am_actions'})
haters_evnt['share'] = haters_evnt['am_actions'] / haters_evnt['am_actions'].sum()
haters_evnt = haters_evnt.sort_values(by='share').reset_index()
def actions_share_plot(df, mode):
    # построим комбинированный график - столбчатые диаграмммы - количество действий действий каждого типа
    # а линия - доля в общем количество действий
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    for i, action in enumerate(df['event_name']):
        fig.add_trace(go.Bar(
            x=[action],
            y=[df.loc[i, 'am_actions']],
            name=action,
            marker=dict(color=px.colors.qualitative.Pastel2[i % len(px.colors.qualitative.Pastel2)]),
            text=[df.loc[i, 'am_actions']],
            textposition='inside'
        ))

    fig.update_layout(
        title=f'Распределение активности юзеров, {mode} рекомендации',
        xaxis=dict(title='название действия в логах'),
        legend_title='название действия',
        showlegend=True
    )

    fig.update_traces(marker_line_color='black', marker_line_width=1)

    fig.add_trace(go.Scatter(
        x=df['event_name'],
        y=df['share'],
        mode='lines',
        name='доля события',
        line=dict(color='red')
    ), secondary_y=True)

    fig.update_yaxes(title_text='кол-во логов', secondary_y=False)
    fig.update_yaxes(title_text='доля в общем кол-ве логов', secondary_y=True)

    fig.show()
# построим комбинированный график для юзеров, использующих рекомендации
actions_share_plot(lovers_evnt, 'использующих')
# построим комбинированный график для юзеров, игнорирующих рекомендации
actions_share_plot(haters_evnt, 'игнорирующих')
Основные различия между пользователями, использующими и игнорирующими рекомендации:

для юзеров, использующих рекомендации, доля "tips_show" в общем кол-ве событий больше на 20%, 71,4% против 50,4%;
для юзеров, игнорирующих рекомендации, вторым наиболее популярным событием в логах, после показа рекомендаций, является просмотр фото с долей 16,2%;
стоит заметить, что топ 5 популярных событий в логах для данных двух сегментов различается.
Для юзеров, использующих рекомендации, он выглядит следующим образом (название события - доля в общем кол-ве логов):

показ рекомендаций - 71,4%;
просмотр контактов - 6,9%;
клик на рекомендацию - 6,5%;
просмотр карты объявлений - 5,3%;
открытие карточки объявления - 5,28%.
А для юзеров, игнорируюищих рекомендации, он выглядит следующим образом:

показ рекомендаций - 50,4%;
просмотр фото - 16,2%;
поисковая активность - 10,4%;
открытие карточки объявления - 8,9%;
просмотр контактов - 5,9%.
Таким образом, можно сказать, что пользователи, использующие рекомендации, быстрее переходят к просмотру контактов - даже без просмотра фото. Возможными причинами таких паттернов в поведении юзеров может быть:

юзеры, использующие реки, проще идут на контакт с продавцами, потому что у них больше свободного времени -> значит больше сделок на таких площадках (по типу Авито) -> проще относятся к коммуникации с продавцами;
наши алгоритмы рекомендательной системы настолько хороши, что юзеры могут даже не смотреть фото в объялении и сразу переходя к просмотру контактов;
Подытоживая данное сравнение, стоит отметить, что данные показывают, что различия в поведении 2-ух сегментов юзеров - есть, но что более важно, для нас, как для сервиса, юзеры, использующие рекомендации, более привлекательны, поскольку конверсия в просмотр контатков выше -> значит при прочих равных условиях вероятность того, что человек приобретет вещь, выше -> следовательно, можно ожидать приток селлеров, а после него и приток новой части покупателей, ведь на площадке стало больше разных категорий товаров.

Анализ источников трафика
# посчитаем среднее количество действий на пользователя по каждому источнику привлечения
avg_actions = mob_df.groupby(['source', 'user_id'], as_index=False) \
                    ['event_time'].count() \
                    .rename(columns={'event_time': 'am_actions'})
for source in avg_actions['source'].unique():
  df_t = avg_actions[avg_actions['source'] == source]['am_actions'].mean()
  print(f'Среднее кол-во действий пользователей из источника {source} = {round(df_t, 2)}')
  print()
Среднее кол-во действий пользователей из источника google = 18.11

Среднее кол-во действий пользователей из источника other = 15.83

Среднее кол-во действий пользователей из источника yandex = 17.73

Поскольку среднее значение в нашем случае не столь репрезентативно, то дале мы рассмотрим количестве действий для 25%, 15%, 10% и 5% самых активных пользователя по каждому источнику привлечения и подведем итоги по каждому каналу привлечения

# посчитаем количество действий для 25% самых активных пользователя по каждому источнику привлечения
for source in avg_actions['source'].unique():
  df_t = avg_actions[avg_actions['source'] == source]['am_actions'].quantile(0.75)
  print(f'25% самых активных пользователей из источника {source} совершают по {round(df_t, 2)} действий')
  print()
25% самых активных пользователей из источника google совершают по 18.0 действий

25% самых активных пользователей из источника other совершают по 16.0 действий

25% самых активных пользователей из источника yandex совершают по 18.0 действий

# посчитаем количество действий для 15% самых активных пользователя по каждому источнику привлечения
for source in avg_actions['source'].unique():
  df_t = avg_actions[avg_actions['source'] == source]['am_actions'].quantile(0.85)
  print(f'15% самых активных пользователей из источника {source} совершают по {round(df_t, 2)} действий')
  print()
15% самых активных пользователей из источника google совершают по 27.8 действий

15% самых активных пользователей из источника other совершают по 26.0 действий

15% самых активных пользователей из источника yandex совершают по 26.0 действий

# посчитаем количество действий для 10% самых активных пользователя по каждому источнику привлечения
for source in avg_actions['source'].unique():
  df_t = avg_actions[avg_actions['source'] == source]['am_actions'].quantile(0.9)
  print(f'10% самых активных пользователей из источника {source} совершают по {round(df_t, 2)} действий')
  print()
10% самых активных пользователей из источника google совершают по 38.0 действий

10% самых активных пользователей из источника other совершают по 34.0 действий

10% самых активных пользователей из источника yandex совершают по 36.0 действий

# посчитаем количество действий для 5% самых активных пользователя по каждому источнику привлечения
for source in avg_actions['source'].unique():
  df_t = avg_actions[avg_actions['source'] == source]['am_actions'].quantile(0.95)
  print(f'5% самых активных пользователей из источника {source} совершают по {round(df_t, 2)} действий')
  print()
5% самых активных пользователей из источника google совершают по 58.6 действий

5% самых активных пользователей из источника other совершают по 52.55 действий

5% самых активных пользователей из источника yandex совершают по 61.35 действий

# строим бокс плот по каждому каналу привлечения
fig = px.box(avg_actions, x='source', y='am_actions', points='outliers', color='source',
             color_discrete_sequence=px.colors.qualitative.Antique)
fig.update_layout(
    title='Распределение количества действий юзеров по каналам привлечения',
    xaxis_title='Источник',
    yaxis_title='Количество действий',
    legend_title='Канал привлечения',
    showlegend=True
)

fig.show()
# строим бокс плот по каждому каналу привлечения
fig = px.box(avg_actions, x='source', y='am_actions', points='outliers', color='source',
             color_discrete_sequence=px.colors.qualitative.Antique)
fig.update_layout(
    title='Распределение количества действий юзеров по каналам привлечения',
    xaxis_title='Источник',
    yaxis_title='Количество действий',
    legend_title='Канал привлечения',
    showlegend=True
)

fig.update_yaxes(range=[0, 50])

fig.show()
Таким образом, можно сказать, что самые активные юзеры приходят либо из "google", либо из "yandex", однако в среднем метрики по 25, 15, 10 и 5 % самых активных юзеров для всех каналов привлечения +/- одинаковые, поэтому нельзя выделить какой-то один канал, из которого приходят юзеры, перекащивающие распределение количества действий.

Далее мы перейдем к визуализации доли целевого трафика по каждому каналу

# посчитаем общее кол-во юзеров по каналам
total = tst.groupby('source', as_index=False)['user_id'].nunique().rename(columns={'user_id': 'total_am'})

# посчитаем кол-во целевых юзеров - просмотревших контакты
# по каждому источнику привлечения
target = tst[tst['is_success'] == True].groupby('source', as_index=False)['user_id'].nunique().rename(columns={'user_id': 'am_target'})

# смержим два дф
merged_df = pd.merge(total, target, on='source')

# посчитаем долю целевого трафика по каждому каналу
merged_df['share_am_target'] = merged_df['am_target'] / merged_df['total_am']
merged_df['share_non_target'] = 1 - merged_df['share_am_target']


#
pie = merged_df.drop(columns=['total_am', 'am_target'])
Мы получили таблицу, в которой по каждому источнику есть кол-во и доля юезров, просмотревших контакты, и тех, кто ни разу не посмотрел контакты продавцов.

Далее мы построим круговую диаграмму для каждого канала привлечения, на которой отобразим долю целевого трафика - людей, просмотревших контакты - и нецелевого.

Доля целевого трафика по каждому каналу
# создаем список с названиями источников для итерирования по нему в цикле
sources = pie['source'].unique()

# задаем размеры сетки с графиками, а также названия графиокв
fig = sp.make_subplots(rows=1, cols=3, subplot_titles=sources, specs=[[{'type': 'pie'}]*3])

# строим графики
for i, source in enumerate(sources):
    df_t = (pie[pie['source'] == source]
            .set_index('source')
            .rename(columns={'share_am_target': 'целевой трафик', 'share_non_target': 'нецелевой трафик'})
            .T)
    labels = df_t.index.values
    values = df_t[source]

    pie_trace = go.Pie(labels=labels, values=values, textinfo='percent',
                       marker=dict(line=dict(color='black', width=1)),
                       pull=[0, 0.3],
                       marker_colors=px.colors.qualitative.Pastel2)

    fig.add_trace(pie_trace, row=1, col=i+1)
    fig.update_layout(title_text='Доли целевого и нецелевого трафика внутри канала привлечения')

fig.update_layout(showlegend=True, legend=dict(orientation='h'))

fig.show()
По графику выше видно, что доля пользователей, просмотревших контакты, больше всего в источнике – "yandex" - 24,7%, чуть меньше в "google" c 24,4%, а наименьшая доля в "other" - лишь 18,5%.

Таким образом, если мы хотим растить долю пользователей, запрашивающих контакты, то в первую очередь стоит смотреть на масштабирование каналов "yandex" или "google", при условии, что стоимость привлечения одинаковая для всех каналов.

Итоги анализа каналов привлечения

Можно сказать, что в целом, все юзеров со всех каналов +/- одинаковые, однако с учетом того, что самые активные юзеры приходят не с источника "other", а с "google" или с "yandex", тем более, в этих 2-ух каналах доля целевого трафика - юзеров, просмотривающих контакты - почти одинаковая;

если выбирать канал для масштабирования, то это опредленно должен быть либо "yandex", либо "google", при условии, что рекламный рынок во всех 3 каналах +/- одинаковый.

Шаг 4. Проработка основных вопросов исследования
Выделение наиболее популярных сценариев использования приложения
# переименуем столбец, что сохранить информацию об источнике трафика
mob_df = mob_df.rename(columns={'source': 'traff_source'})
Начнем с определения наиболее популярных сценариев. Для решения данной задачи была прочитана статья на хабре - https://habr.com/en/articles/566568/ - и код из статьи использовался в качестве шаблона, однако на этапе применения функции "add_features" добавляем условие на удаление строк, в которых совпадает "source" и "target", а также немного оптимизируем процесс удаления строк, в которых юзеры совершили более 4 действий за сессию. Раньше мы просто удаляли записи со значением в столбце "step" больше 4, а теперь мы будем удалять все записи, относящиеся к таким сессиям - для этого напишем функцию drop_sessions

def add_features(df):
    sorted_df = df.sort_values(by=['session_id', 'event_time'])
    sorted_df['step'] = sorted_df.groupby('session_id').cumcount() + 1
    sorted_df['source'] = sorted_df['event_name']
    sorted_df['target'] = sorted_df.groupby('session_id')['source'].shift(-1)

    # пропишем условие на сохранение строки с уникальными значениями в столбце 'target' для каждой сессии
    unique_steps_df = sorted_df.drop_duplicates(subset=['session_id','target'], keep='first')

    return unique_steps_df.drop(['event_name'], axis=1)
# применим вышеописанную функцию к нашему дф
table = add_features(mob_df)
# применим условие к полученной таблице
new_df = table[table['source'] != table['target']]
new_df
event_time	user_id	traff_source	session_id	step	source	target
839	2019-10-07 13:49:42	0001b1d5-b74a-4cbf-aeb0-7df5947bf349	other	1	9	tips_show	NaN
6546	2019-10-09 18:35:28	0001b1d5-b74a-4cbf-aeb0-7df5947bf349	other	2	2	map	tips_show
6566	2019-10-09 18:42:23	0001b1d5-b74a-4cbf-aeb0-7df5947bf349	other	2	4	tips_show	NaN
36416	2019-10-21 19:53:17	0001b1d5-b74a-4cbf-aeb0-7df5947bf349	other	3	2	tips_show	map
36486	2019-10-21 20:07:30	0001b1d5-b74a-4cbf-aeb0-7df5947bf349	other	3	14	tips_show	NaN
...	...	...	...	...	...	...	...
72321	2019-11-03 14:32:56	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10632	1	tips_show	contacts_show
72325	2019-11-03 14:33:48	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10632	2	contacts_show	tips_show
72364	2019-11-03 14:48:44	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10632	15	tips_show	NaN
72547	2019-11-03 15:47:44	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10633	5	tips_show	contacts_show
72689	2019-11-03 16:08:25	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10633	14	tips_show	NaN
19291 rows × 7 columns

Что получили в итоговой таблице:

события каждого id отсортированы по времени;
созданы пары событий source - target;
добавлен шаг между этими событиями для построения диаграммы;
удален столбец event_name, так как в дальнейших преобразованиях он использоваться не будет.
Далее мы определим, сколько шагов (этапов) лучше всего выбрать для нашего CJM - то есть мы посмотрим на основную стат. информацию по столбцу с кол-во шагов в сессии

# выводим основную стат. информацию по столбцу с кол-во шагов в сессии
new_df['step'].describe()
count    19291.000000
mean         5.405681
std          7.813883
min          1.000000
25%          1.000000
50%          3.000000
75%          6.000000
max        139.000000
Name: step, dtype: float64
Таким образом, оптимальнее всего будет выбрать 4 шагов, поскольку свыше этого значения, лежат сессии меньшей части пользователей (менее 50%)

def drop_sessions(df):
    session_max_step = df.groupby('session_id')['step'].max()
    sessions_to_drop = session_max_step[session_max_step > 4].index
    return df[~df['session_id'].isin(sessions_to_drop)].reset_index(drop=True)

# применим фнукцию, которая исключит сессии, в которых больше 4 шагов - сессии меньшей части юзеров
df_comp = drop_sessions(new_df)
df_comp
event_time	user_id	traff_source	session_id	step	source	target
0	2019-10-09 18:35:28	0001b1d5-b74a-4cbf-aeb0-7df5947bf349	other	2	2	map	tips_show
1	2019-10-09 18:42:23	0001b1d5-b74a-4cbf-aeb0-7df5947bf349	other	2	4	tips_show	NaN
2	2019-10-20 20:04:17	00157779-810c-4498-9e05-a1e9e3cedf93	yandex	7	2	photos_show	contacts_show
3	2019-10-20 20:05:36	00157779-810c-4498-9e05-a1e9e3cedf93	yandex	7	4	photos_show	NaN
4	2019-11-03 17:12:10	00157779-810c-4498-9e05-a1e9e3cedf93	yandex	11	1	contacts_show	NaN
...	...	...	...	...	...	...	...
7775	2019-11-02 18:01:27	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10630	1	tips_show	contacts_show
7776	2019-11-02 18:17:41	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10630	2	contacts_show	NaN
7777	2019-11-02 19:25:54	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10631	1	tips_show	contacts_show
7778	2019-11-02 19:26:08	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10631	2	contacts_show	tips_show
7779	2019-11-02 19:30:50	fffb9e79-b927-4dbb-9b48-7fd09b23a62b	google	10631	4	tips_show	NaN
7780 rows × 7 columns

def get_source_index(df):

    """Функция генерации индексов source

    Args:
        df (pd.DataFrame): исходная таблица с признаками step, source, target.
    Returns:
        dict: словарь с индексами, именами и соответсвиями индексов именам source.
    """

    res_dict = {}

    count = 0
    # получаем индексы источников
    for no, step in enumerate(df['step'].unique().tolist()):
        # получаем уникальные наименования для шага
        res_dict[no+1] = {}
        res_dict[no+1]['sources'] = df[df['step'] == step]['source'].unique().tolist()
        res_dict[no+1]['sources_index'] = []
        for i in range(len(res_dict[no+1]['sources'])):
            res_dict[no+1]['sources_index'].append(count)
            count += 1

    # соединим списки
    for key in res_dict:
        res_dict[key]['sources_dict'] = {}
        for name, no in zip(res_dict[key]['sources'], res_dict[key]['sources_index']):
            res_dict[key]['sources_dict'][name] = no
    return res_dict
# создаем словарь
source_indexes = get_source_index(df_comp)
def generate_random_color():

    """Случайная генерация цветов rgba

    Args:

    Returns:
        str: Строка со сгенерированными параметрами цвета
    """

    # сгенерим значение для каждого канала
    r, g, b = np.random.randint(255, size=3)
    return f'rgba({r}, {g}, {b}, 1)'
def colors_for_sources(mode):

    """Генерация цветов rgba

    Args:
        mode (str): сгенерировать случайные цвета, если 'random', а если 'custom' -
                    использовать заранее подготовленные
    Returns:
        dict: словарь с цветами, соответствующими каждому индексу
    """
    # словарь, в который сложим цвета в соответствии с индексом
    colors_dict = {}

    if mode == 'random':
        # генерим случайные цвета
        for label in df_comp['source'].unique():
            r, g, b = np.random.randint(255, size=3)
            colors_dict[label] = f'rgba({r}, {g}, {b}, 1)'

    elif mode == 'custom':
        # присваиваем ранее подготовленные цвета
        colors = requests.get('https://raw.githubusercontent.com/rusantsovsv/senkey_tutorial/main/json/colors_senkey.json').json()
        for no, label in enumerate(df_comp['source'].unique()):
            colors_dict[label] = colors['custom_colors'][no]

    return colors_dict
# генерим цвета из своего списка
colors_dict = colors_for_sources(mode='random')
def percent_users(sources, targets, values):

    """
    Расчет уникальных id в процентах (для вывода в hover text каждого узла)

    Args:
        sources (list): список с индексами source.
        targets (list): список с индексами target.
        values (list): список с "объемами" потоков.

    Returns:
        list: список с "объемами" потоков в процентах
    """

    # объединим источники и метки и найдем пары
    zip_lists = list(zip(sources, targets, values))

    new_list = []

    # подготовим список словарь с общим объемом трафика в узлах
    unique_dict = {}

    # проходим по каждому узлу
    for source, target, value in zip_lists:
        if source not in unique_dict:
            # находим все источники и считаем общий трафик
            unique_dict[source] = 0
            for sr, tg, vl in zip_lists:
                if sr == source:
                    unique_dict[source] += vl

    # считаем проценты
    for source, target, value in zip_lists:
        new_list.append(round(100 * value / unique_dict[source], 1))

    return new_list
def lists_for_plot(source_indexes=source_indexes, colors=colors_dict, frac=10):

    """
    Создаем необходимые для отрисовки диаграммы переменные списков и возвращаем
    их в виде словаря

    Args:
        source_indexes (dict): словарь с именами и индексами source.
        colors (dict): словарь с цветами source.
        frac (int): ограничение на минимальный "объем" между узлами.

    Returns:
        dict: словарь со списками, необходимыми для диаграммы.
    """

    sources = []
    targets = []
    values = []
    labels = []
    link_color = []
    link_text = []

    # проходим по каждому шагу
    for step in tqdm(sorted(df_comp['step'].unique()), desc='Шаг'):
        if step + 1 not in source_indexes:
            continue

        # получаем индекс источника
        temp_dict_source = source_indexes[step]['sources_dict']

        # получаем индексы цели
        temp_dict_target = source_indexes[step+1]['sources_dict']

        # проходим по каждой возможной паре, считаем количество таких пар
        for source, index_source in tqdm(temp_dict_source.items()):
            for target, index_target in temp_dict_target.items():
                # делаем срез данных и считаем количество id
                temp_df = df_comp[(df_comp['step'] == step)&(df_comp['source'] == source)&(df_comp['target'] == target)]
                value = len(temp_df)
                # проверяем минимальный объем потока и добавляем нужные данные
                if value > frac:
                    sources.append(index_source)
                    targets.append(index_target)
                    values.append(value)
                    # делаем поток прозрачным для лучшего отображения
                    link_color.append(colors[source].replace(', 1)', ', 0.2)'))

    labels = []
    colors_labels = []
    for key in source_indexes:
        for name in source_indexes[key]['sources']:
            labels.append(name)
            colors_labels.append(colors[name])

    # посчитаем проценты всех потоков
    perc_values = percent_users(sources, targets, values)

    # добавим значения процентов для howertext
    link_text = []
    for perc in perc_values:
        link_text.append(f"{perc}%")

    # возвратим словарь с вложенными списками
    return {'sources': sources,
            'targets': targets,
            'values': values,
            'labels': labels,
            'colors_labels': colors_labels,
            'link_color': link_color,
            'link_text': link_text}
# создаем словарь
data_for_plot = lists_for_plot()
Шаг:   0%|          | 0/4 [00:00<?, ?it/s]
  0%|          | 0/9 [00:00<?, ?it/s]
  0%|          | 0/9 [00:00<?, ?it/s]
  0%|          | 0/8 [00:00<?, ?it/s]
def plot_senkey_diagram(data_dict=data_for_plot):

    """
    Функция для генерации объекта диаграммы Сенкей

    Args:
        data_dict (dict): словарь со списками данных для построения.

    Returns:
        plotly.graph_objs._figure.Figure: объект изображения.
    """

    fig = go.Figure(data=[go.Sankey(
        domain = dict(
          x =  [0,1],
          y =  [0,1]
        ),
        orientation = "h",
        valueformat = ".0f",
        node = dict(
          pad = 50,
          thickness = 15,
          line = dict(color = "black", width = 0.1),
          label = data_dict['labels'],
          color = data_dict['colors_labels']
        ),
        link = dict(
          source = data_dict['sources'],
          target = data_dict['targets'],
          value = data_dict['values'],
          label = data_dict['link_text'],
          color = data_dict['link_color']
      ))])
    fig.update_layout(title_text="Sankey Diagram", font_size=10, width=1050, height=700)

    # возвращаем объект диаграммы
    return fig
# сохраняем диаграмму в переменную
senkey_diagram = plot_senkey_diagram()
senkey_diagram.show()
# обновим фнукцию, написанную выше, для создания дф
# для анализа прошедшего времени между "tips_click" и "contacts_show"
def add_features2(df):
    sorted_df = df.sort_values(by=['session_id', 'event_time'])
    sorted_df['step'] = sorted_df.groupby('session_id').cumcount() + 1
    sorted_df['source'] = sorted_df['event_name']
    sorted_df['target'] = sorted_df.groupby('user_id')['source'].shift(-1)

    return sorted_df
sessions_df = add_features2(mob_df)
Переходим к анализу сессий, в которых "contacts_show" было совершено вторым действием

# напишем функцию для построения воронок для двухэтапных сессий
def generate_funnel_plot(actions, sessions_df):
    base_event_df = (mob_df[mob_df['event_name'] == actions]
                     .groupby('user_id', as_index=False)
                     .agg(base_event_time=('event_time', 'min')))

    base_event_users = base_event_df['user_id'].nunique()

    contacts_user_df = (mob_df[mob_df['event_name'] == 'contacts_show']
                        .merge(base_event_df, on='user_id', how='left'))

    contacts_user_df = contacts_user_df[contacts_user_df['event_time'] > contacts_user_df['base_event_time']]

    target_event_users = contacts_user_df['user_id'].nunique()

    fig = go.Figure()
    fig.add_trace(go.Funnel(
        y=[actions, 'contacts_show'],
        x=[base_event_users, target_event_users],
        textposition="inside",
        textinfo="value+percent initial+percent previous",
        name=f"{actions} Funnel"
    ))

    fig.update_layout(title=f'Двухэтапная воронка с началом в {actions}', showlegend=False)

    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )

    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)

    fig.show()
# построим воронку с началом в "tips_show"
generate_funnel_plot('tips_show', sessions_df)
# построим воронку с началом в "search"
generate_funnel_plot('search', sessions_df)
# построим воронку с началом в "photos_show"
generate_funnel_plot('photos_show', sessions_df)
Таким образом, по воронкам выше можно понять, что среди юзеров, совершивших целевое действие на втором шаге (второй лог внутри сессии) - просмотр контактов, наиболее популярным сценарием использования нашего приложения является переход от показа рекомендаций ("tips_show"). У 2801 нашего юзера сессия начиналась с этого события, и лишь 495 далее просмотрели контакты - конверсия в целевое действие составила 17,7% на 2 этапе. Следующим по популярности сцеанрием является переход от поисковой активности ("search") к просмотру контактов - из 1666 юзеров, у которых сессии начались с поиска, целевое действие совершили 342 пользователя, то есть конверсия в просмотр контактов на втором шаге в данном сценарии - 20,5%. И третим по популярности сценарием является переход от просмотра фото ("photos_show") к запросу контактов. Так, из 1095 юзеров просмотрели контакты 301 пользователь - конверсия целых 27,5%.

Подводя итог анализу двухэтапных пользовательских сценариев, стоит отметить следующие моменты:

чем более популярен сценрий - тем меньше конверсия в целевое действие (варьируется от 17,7% до 27,5%);
связать данную тенденцию можно с тем, что чем более популярен сценарий, тем менее ярко выражен запрос у покупателя на поиск какого-либо товара - это подтверждается и событием, с которого начинаются данные воронки;
самый популярный, но с наименьшей конверсией - сценарий, начинающийся с рекомендаций;
наименее популярный, но с наибольшей конверсией - переход от просмотра фото к запросу контактов.
Таким образом, в этой ситуации можно было сделать ошибочный вывод - если мы сможем сконвертить юзеров в просмотр фото после клика на рекомендацию, то мы сможем увеличить конверсию в просмотр контатков. Но дело в том, что люди, которые смотрят фото, скорее всего, совершают данное действие поскольку у них "болит" потребность в данной услуге / товаре, а вот юзеры, кликающие на реки, могут просто сидеть в ленте рекомендаций, даже не имея цели что-то приобрести. Но данные предположения мы сможем проверить лишь с помощью проведения глубинных (пользовательских) интервью.

# определим ай ди сессий, в которых кол-во шагов было 1 или 2
two_steps_sess = sessions_df[sessions_df['step'] < 3]

# определим события, из которых мы считаем конверсию в целевое действие - "contacts_show"
actions = ['tips_show', 'search', 'photos_show']

# пишем цикл с подсчетом общего кол-ва сессий с каждым действием из списка
# а также считаем кол-во сессий с целевым действие и конверсию
for action in actions:
  total_sess = sessions_df[sessions_df['source'] == action]['session_id'].nunique()
  target_sess = two_steps_sess[(two_steps_sess['source'] == action)
             & (two_steps_sess['target'] == 'contacts_show')]['session_id'].nunique()
  cr = round(target_sess / total_sess, 4)
  print(f'Общее количество сессий с действием {action} составило {total_sess}')
  print(f'Всего {target_sess} сессий с действием {action}, которые закончились целевым действие')
  print(f'Конверсия в целевое действие составила {cr*100}%')
  print()
Общее количество сессий с действием tips_show составило 6189
Всего 348 сессий с действием tips_show, которые закончились целевым действие
Конверсия в целевое действие составила 5.62%

Общее количество сессий с действием search составило 2997
Всего 146 сессий с действием search, которые закончились целевым действие
Конверсия в целевое действие составила 4.87%

Общее количество сессий с действием photos_show составило 2571
Всего 159 сессий с действием photos_show, которые закончились целевым действие
Конверсия в целевое действие составила 6.18%

Переходим к анализу сессий, в которых "contacts_show" было совершено третьим действием и на этом мы закончим детальное рассмотрение сессий

# создадим функцию для построения воронок для сессий, в которых
# целевое действие - "contacts_show" - было совершено на 3 шаге
def three_steps_funnel_plot(mid_action, sources_mid_act, sessions_df):
    # рисуем сетку для будущих графиков
    fig = make_subplots(rows=1, cols=len(sources_mid_act), subplot_titles=sources_mid_act, horizontal_spacing=0.2)

    # подготавливаем данные для будущих графиков
    for i, source_mid_action in enumerate(sources_mid_act, start=1):
        total_usr_id = sessions_df[sessions_df['source'] == source_mid_action]['user_id'].unique()
        mid_act = sessions_df[(sessions_df['user_id'].isin(total_usr_id)) &
                              (sessions_df['source'] == mid_action)]['user_id'].unique()
        target_am = sessions_df[(sessions_df['user_id'].isin(mid_act)) &
                                (sessions_df['event_name'] == 'contacts_show')]['user_id'].nunique()

        # создаем воронку
        fig.add_trace(go.Funnel(
            y=[source_mid_action, mid_action, 'contacts_show'],
            x=[len(total_usr_id), len(mid_act), target_am],
            textposition="inside",
            textinfo="value+percent initial+percent previous",
            name=f"{source_mid_action} Funnel"
        ), row=1, col=i)

        # меняем цвет бэкграунда
        fig.update_layout(
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)'
        )

        # убираем сетку
        fig.update_xaxes(showgrid=False)
        fig.update_yaxes(showgrid=False)

    fig.update_layout(title='Трехэтапные воронки', showlegend=False)

    fig.show()
# создадим список источников для промежуточного действия "tips_show"
sources_mid_act = ['map', 'advert_open', 'search']
# построим воронку с промежуточным действием "tips_show"
three_steps_funnel_plot('tips_show', sources_mid_act, sessions_df)
# создадим список источников для промежуточного действия "photos_show"
sources_mid_act = ['advert_open', 'search']
# построим воронку с промежуточным действием "photos_show"
three_steps_funnel_plot('photos_show', sources_mid_act, sessions_df)
Переходя к анализу трехэтапных воронок, можно заметить, что конверсия в целевое действие по всем сценариям сильно ниже по сравнению с двуххэтапными сценариями.

Так среди юзеров, совершивших целевое действие - просмотр контактов - на третьем шаге (третий лог внутри сессии) после просмотра рекомендаций (второй лог) наиболее популярным сценарием использования нашего приложения является переход от поиска ("search"). У 1666 наших юзеров сессия начиналась с этого события, из них 801 просмотрели рекомендации, а до просмотра контактов дошли лищь 139 пользователей. То есть конверсия в целевое действие составила 8,3% на 3 шаге. Следующим по популярности сцеанрием является переход от просмтотра карты ("map") к просмотру рекомендаций - из 1456 юзеров, у которых сессии начались с открытия карты, далее перешло к просмотру рекомендаций 1352 пользователя, а из них 275 перешли к просмотру контактов, то есть конверсия в целевое действие на третьем шаге в данном сценарии - 18,9%. И третим по популярности сценарием является переход от просмотра объявлений ("advert_open"), далее просмотор рек, а потом уже запрос контактов. Так, из 751 юзеров, открывших объявление, к просмотру рекомендаций перешло 590 человек, из них 88 просмотрели контакты - конверсия 11,7%.

Переходим к воронкам, у которых промежуточным действием было просмотр фото, тут мы рассмотрим 2 сценария. Так, в первом случае из 1666 наших юзеров, начавших с поиска, 647 просмотрели фото, а до просмотра контактов дошли 195 пользователей. То есть конверсия в целевое действие составила 11,5% на 3 шаге. Во втором случае из 751 юзера, открывших объявление, к просмотру фото перешло лишь 73 человека, а до просмотра контактов дошли всего лишь 38 юзеров - то есть конверсия данного сценария - 5,3%.

Подводя итог анализу трехэтапных пользовательских сценариев, стоит отметить следующие моменты:

в данном случае у нас нет тендции к тому, чтобы сценарии с наименьшей базой юезров на первом этапе получали наибольшую конверсию в целевое действие в итоге;
наиболее популярным сценарием в данном случае является тот, что начинается с поисковой активности, если смотреть по началу воронки;
а вот наибольшей популярностью с точки зрения количества юзеров, совершивших целевое действие, является сценарий, начавшийся с просмотра карты;
так, в случае со сценарием "map" -> "photos_show" -> "contacts_show" конверсия в целевое действие составила целых 18,9% - что является наибольшим значением, среди всех сценариев, в которых контакты просмотрели на 3 шаге;
наименее популярный сценарий - "advert_open" -> "photos_show" -> "contacts_show" - 38 юзеров совершили целевое действие или же лишь 5,3% от начала воронки.
Выводом, обобщающим анализ пользовательских сценариев, может стать факт того, что чем меньше шагов до целевого события, тем выше конверсия в его совершение. Как это можно применить? Как мне кажется, проведение пользовательских интервью для каждого сегмента юзеров - тех, кто обычно приходит в приложение с конкретным запросом (сессии начинаются с "search"); тех, кто готов полистать наши рекомендации (сессии начинаются с "tips_show" или содержат много данных логов внутри сессии) и для остальных сегментов, рассмотренных выше - что позволит спроектировать сценарии использования с учетом особенностей каждого сегмента, что в конечном итоге может приветси к сокращению средней длины пути до целевого действия.

# создадим переменную с сессиями до 3-х шага
three_steps_sess = sessions_df[sessions_df['step'] < 4]

# напишем функцию, основанную на идеи выше, для расчета конверсии в целевое действие по сессиям
# для 3 этапных сессий, в которых переход к просмотру контактов произошел от показа рекомендации
def three_steps_cr(mid_action, action):
  act_sess = three_steps_sess[(three_steps_sess['step'] == 1) &
                              (three_steps_sess['source'] == action)]['session_id'].unique()
  act_target_sess = (three_steps_sess[three_steps_sess['session_id'].isin(act_sess)]
                                    [((three_steps_sess['step'] == 3) &
                                      (three_steps_sess['source'] == mid_action) &
                                    (three_steps_sess['target'] == 'contacts_show'))]['session_id'].nunique())
  act_cr = round(act_target_sess / len(act_sess), 4)
  print(f'Общее количество сессий с действием {action} составило {len(act_sess)}')
  print(f'Всего {act_target_sess} сессий с действием {action}, которые закончились целевым действие')
  print(f'Конверсия в целевое действие составила {act_cr*100}%')
# Считаем конверсию для сессий, начавшихся с "map"
# и промежуточным действием "tips_show"
three_steps_cr('tips_show', 'map')
Общее количество сессий с действием map составило 1306
Всего 20 сессий с действием map, которые закончились целевым действие
Конверсия в целевое действие составила 1.53%
# Считаем конверсию для сессий, начавшихся с "advert_open"
# и промежуточным действием "tips_show"
three_steps_cr('tips_show', 'advert_open')
Общее количество сессий с действием advert_open составило 425
Всего 2 сессий с действием advert_open, которые закончились целевым действие
Конверсия в целевое действие составила 0.47000000000000003%
# Считаем конверсию для сессий, начавшихся с "search"
# и промежуточным действием "tips_show"
three_steps_cr('tips_show', 'search')
Общее количество сессий с действием search составило 2517
Всего 9 сессий с действием search, которые закончились целевым действие
Конверсия в целевое действие составила 0.36%
# Считаем конверсию для сессий, начавшихся с "advert_open"
# и промежуточным действием "photos_show"
three_steps_cr('photos_show', 'advert_open')
Общее количество сессий с действием advert_open составило 425
Всего 1 сессий с действием advert_open, которые закончились целевым действие
Конверсия в целевое действие составила 0.24%
# Считаем конверсию для сессий, начавшихся с "search"
# и промежуточным действием "photos_show"
three_steps_cr('photos_show', 'search')
Общее количество сессий с действием search составило 2517
Всего 14 сессий с действием search, которые закончились целевым действие
Конверсия в целевое действие составила 0.5599999999999999%
Насколько хороши наши рекомендации?
Проверка гипотезы: "конверсия в просмотры контактов различается у юзеров, совершающих действия tips_show и tips_click, отличается от — только tips_show"
# выделяем айди юзеров, совершающих "tips_show"
tips_show_usr = mob_df[mob_df['event_name'] == 'tips_show']['user_id'].unique()

# выделяем айди юзеров, совершающих "tips_show" и "tips_click"
tips_usr = (mob_df[(mob_df['event_name'] == 'tips_click') &
                   (mob_df['user_id'].isin(tips_show_usr))])

# выделяем айди юзеров, которые посмотрели контакты и совершали  "tips_show" и "tips_click"
click_contacts = mob_df[(mob_df['user_id'].isin(tips_usr['user_id'])) &
                             (mob_df['event_name'] == 'contacts_show')]

# выделяем айди юзеров, совершающих только "tips_show"
# и исключаем пересечения с теми, кто совершает "tips_show" и "tips_click"
no_tips_usr = (mob_df[(mob_df['event_name'] == 'tips_show') &
                      (~mob_df['user_id'].isin(tips_usr['user_id']))])

# выделяем среди юзеров, совершающих только "tips_show", тех, кто посмотрел контакты
no_tips_contacts = mob_df[(mob_df['user_id'].isin(no_tips_usr['user_id'])) &
                             (mob_df['event_name'] == 'contacts_show')]
# считаем конверсию в целевое действие у юзеров, использующих реки
click_contacts['user_id'].nunique() / tips_usr['user_id'].nunique()
0.3063973063973064
# считаем конверсию в целевое действие у юзеров, не использующих реки
no_tips_contacts['user_id'].nunique() / no_tips_usr['user_id'].nunique()
0.16972843450479233
Как можно заметить, конверсия в просмотр контактов разная, однако без проведения стат. теста мы не можем с уверенностью утверждать, что мы имеем стат. значимое различие, так что переходим к проведению стат. теста

# добавим в дф с юзерами, использующими рекомендации,
# столбец по которому мы будем проводить стат тест
tips_usr['is_success'] = tips_usr['user_id'].isin(click_contacts['user_id'])

# добавим в дф с юзерами, не использующими рекомендации,
# столбец, по которому мы будем проводить стат тест
no_tips_usr['is_success'] = no_tips_usr['user_id'].isin(no_tips_contacts['user_id'])
H0: конверсия в просмотр контактов для двух сегментов одинаковая

H1: конверсия значение просмотр контактов для двух сегментов разная

# задаем уровень стат. значимости
alpha = .05

# проводим стат. тест о равенстве средних
results = st.ttest_ind(tips_usr['is_success'],
                       no_tips_usr['is_success'], equal_var=False)

# выводим получившееся значение p-value и результаты стат. теста
print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')
p-значение: 8.843583740520597e-12
Отвергаем нулевую гипотезу
P-value < 0.05, а значит мы отвергаем нулевую гипотезу и приримаем альтернативную (H1) - "конверсия в просмотр контактов для двух сегментов разная"

Таким образом, мы можем заметить, что нет большой разницы между конверсией, расчитанной по сессимя и по юзерам. Для юзеров, использующих рекомендации, конверсия в просмотр контактов, расчитанная по уникам (юзерам), равна 30,64%, что больше на 8,87% чем расчитанная по сессиям, а вот для юзеров, игнорирующих рекомендации, составила 16,97% - больше лишь на 0,72%. Возможно, это связано с тем, что юзеры, кликающие на рекомендации, в среднем совершают больше действий за сессию, поэтому конверсия в контакт выше - ведь при прочих равных вероятность того, что они найдут нужный им товар - тоже выше. Далее мы это и проверим

# считаем среднее значение количества действий в сессии для юзеров, использующих рекомендации
mob_df[mob_df['user_id'].isin(tips_usr['user_id'])].groupby(['user_id', 'session_id'])['event_time'].count().mean()
8.758571428571429
# считаем медианное значение количества действий в сессии для юзеров, использующих рекомендации
mob_df[mob_df['user_id'].isin(tips_usr['user_id'])].groupby(['user_id', 'session_id'])['event_time'].count().median()
5.0
# считаем среднее значение количества действий в сессии для юзеров, не использующих рекомендации
mob_df[mob_df['user_id'].isin(no_tips_usr['user_id'])].groupby(['user_id', 'session_id'])['event_time'].count().mean()
8.315719947159842
# считаем медианное значение количества действий для юзеров, не использующих рекомендации
mob_df[mob_df['user_id'].isin(no_tips_usr['user_id'])].groupby(['user_id', 'session_id'])['event_time'].count().median()
5.0
Юзеры, кликающие на рекомендации, в среднем совершают столько же действий, что и те, кто их игнорирует. А медианное значение равно для этих 2 сегментов. Можно сказать, что дело явно не в этом, возможно, стоит смотреть данные не внутри сессий, а в среднем по пользователям - далее мы посмотрим среднее и медианное кол-во действий на юзеров в разрезе данных сегментов.

# считаем среднее значение количества действий для юзеров, использующих рекомендации
mob_df[mob_df['user_id'].isin(tips_usr['user_id'])].groupby('user_id')['event_time'].count().mean()
41.28619528619529
# считаем медианное значение количества действий для юзеров, использующих рекомендации
mob_df[mob_df['user_id'].isin(tips_usr['user_id'])].groupby('user_id')['event_time'].count().median()
20.0
# считаем среднее значение количества действий для юзеров, игнорирующих рекомендации
mob_df[mob_df['user_id'].isin(no_tips_usr['user_id'])].groupby('user_id')['event_time'].count().mean()
17.597843450479232
# считаем медианное значение количества действий для юзеров, игнорирующих рекомендации
mob_df[mob_df['user_id'].isin(no_tips_usr['user_id'])].groupby('user_id')['event_time'].count().median()
10.0
По этим данным различия уже есть. Так метрики юзеров, использующих рекомендации, почти в 2 раза выше. Среднее значением больше более чем в раза - 39 действий против 17.6, а медианное - 18 против 10. Таким образом, можно предположить, что большая конверсия в целевое действие у сегмента, использующего рекомендации, по причине того, что они совершают больше действий, что в свою очередь повышает вероятность того, что они свяжутся с продавцом - это мы и видим в данных. Однако момент в том, что внутри сессии активность юзеров в среднем одинаковая, дело в количестве сессий? Далее мы это и проверим

# считаем среднее значение количества сессий для юзеров, использующих рекомендации
(mob_df[mob_df['user_id'].isin(tips_usr["user_id"])]
       .groupby('user_id')
       ['session_id'].nunique().mean())
4.713804713804714
# считаем среднее значение количества сессий для юзеров, игнорирующих рекомендации
(mob_df[mob_df['user_id'].isin(no_tips_usr["user_id"])]
       .groupby('user_id')
       ['session_id'].nunique().mean())
2.116214057507987
Подводя итог всему выше сказанному, можно выделить следующие моменты:

конверсия в просмотр контактов для юзеров, просматривающих рекомендации (совершают действия 'tips_show' и 'tips_click') и игнорирующих их (совершают только 'tips_show'), разная;
в среднем внутри сессии активность юзеров из данных сегментов одинаковая;
а вот среднее количество сессий на юзера, использующего рекомендации, превышает количетсво сессий юзеров, игнорирующего их, более чем в 2 раза.
Далее мы посмотрим на то, какие действий совершают юзеры из этих 2-ух сегментов.

Расчет времени между событиями advert_open -> contacts_show и tips_click -> contacts_show
# обновим фнукцию, написанную выше, для создания дф
# для анализа прошедшего времени между "tips_click" и "contacts_show"
def add_features2(df):
    sorted_df = df.sort_values(by=['session_id', 'event_time'])
    sorted_df['step'] = sorted_df.groupby('session_id').cumcount() + 1
    sorted_df['source'] = sorted_df['event_name']
    sorted_df['target'] = sorted_df.groupby('user_id')['source'].shift(-1)

    return sorted_df
# создадим копию нашего дф с информацией по сессиям и путем каждой сессии
rec = add_features2(mob_df)
# оставим те сессии юзеров, использующих рекомендации, в которых они просмотрели контакты
rec = rec[rec['session_id'].isin(tips_contacts_session)]
# оставим те строчки, где события "tips_click" или "contacts_show" - между которыми мы считаем время
rec = rec[rec['event_name'].isin(['tips_click', 'contacts_show'])]
# удалим дубли событий, оставив первые по времени появления в логах, внутри каждой сессии
rec = rec.drop_duplicates(subset=['session_id', 'source'], keep='first')
# осортируем дф
rec.sort_values(['session_id', 'event_time'], inplace=True)
# добавляем столбец со времени совершения первого нужного нам действия - "tips_click"
rec['base_event_time'] = rec[rec['event_name'] == 'tips_click']['event_time']
# заполним пропуски в столбце "base_event_time" в строках, где у нас действие не "tips_click"
rec['base_event_time'] = rec.groupby('session_id')['base_event_time'].fillna(method='ffill').fillna(method='bfill')
# заполним пропуски в столбце "base_event_time" в строках, где у нас действие не "tips_click"
rec = rec[rec['event_time'] > rec['base_event_time']]
# считаем разницу во времени между событиями "tips_click" и "contacts_show"
rec['time_diff'] = rec['event_time'] - rec['base_event_time']
# переформируем порядок и количество столбцов в дф
rec = rec.iloc[:, [1, 0, 8, 9, 4]]
# выводим среднее значение прошедшего времени между событиями "tips_click" и "contacts_show"
rec['time_diff'].mean()
Timedelta('0 days 00:12:41.285714285')
Таким образом, мы получили, что после клика на рекомендацию проходит в среднем 12 минут 42 секунд прежде, чем юзеры просмотрят контакты. Не так уж мало времени, возможно, все это время пользователи рассматривают фото или же у нас есть случаи, крайне быстрого и медленного перехода от просмотра объявления к просмотру контактов, давайте посмотрем больше стат. инфы по столбцу с разницей времени между событиями.

# выводим стат. информацию по столбцу со временем между нашими событиями
rec['time_diff'].describe()
count                           28
mean     0 days 00:12:41.285714285
std      0 days 00:12:14.725237488
min                0 days 00:00:18
25%                0 days 00:03:55
50%                0 days 00:07:51
75%         0 days 00:18:50.750000
max                0 days 00:50:48
Name: time_diff, dtype: object
В нашем случае более корректно смотреть на медианное значение, ведь у нас есть четкое понимание того, что распределение наших наблюдение имеет положительную ассиметрию - правосторонний скос. Следовательно, мы знаем, что среднее значение искусственно завышает исследуемый показатель.

Таким образом, возьмем 7 минут 51 секунд за бенчмарк прошедшего времени от клика на рекомендацию до просмотра контактов.

# выделяем сессии юзеров, в которых было действие "advert_open"
advert_session = mob_df.query('event_name == "advert_open"')['session_id'].unique()

# выделяем сессии, в которых было действие "advert_open" и юзеры посмотрели контакты
advert_contacts_session = set(advert_session) & set(contacts_session)
# создадим копию нашего дф с информацией по сессиям и путем каждой сессии
advert = add_features2(mob_df)
# оставим те сессии юзеров, использующих рекомендации, в которых они просмотрели контакты
advert = advert[advert['session_id'].isin(advert_contacts_session)]
# оставим те строчки, где события "advert_open" или "contacts_show" - между которыми мы считаем время
advert = advert[advert['event_name'].isin(['advert_open', 'contacts_show'])]
# удалим дубли событий, оставив первые по времени появления в логах, внутри каждой сессии
advert = advert.drop_duplicates(subset=['session_id', 'source'], keep='first')
# осортируем дф
advert.sort_values(['session_id', 'event_time'], inplace=True)
# добавляем столбец со времени совершения первого нужного нам действия - "advert_open"
advert['base_event_time'] = advert[advert['event_name'] == 'advert_open']['event_time']
# заполним пропуски в столбце "base_event_time" в строках, где у нас действие не "advert_open"
advert['base_event_time'] = advert.groupby('session_id')['base_event_time'].fillna(method='ffill').fillna(method='bfill')
# заполним пропуски в столбце "base_event_time" в строках, где у нас действие не "advert_open"
advert = advert[advert['event_time'] > advert['base_event_time']]
# считаем разницу во времени между событиями "advert_open" и "contacts_show"
advert['time_diff'] = advert['event_time'] - advert['base_event_time']
# переформируем порядок и количество столбцов в дф
advert = advert.iloc[:, [1, 0, 8, 9, 4]]
# выводим среднее значение прошедшего времени между событиями "advert_open" и "contacts_show"
advert['time_diff'].mean()
Timedelta('0 days 00:07:24.151515151')
В среднем от открытия карточки объявления до просмотра контактов проходит 7 минут 24 секнуды - меньше почти на 5,5 минут по сравнению с кликом на рекомендацию. Далее посмотрим больше стат. инфы по столбцу с разницей времени между событиями.

# выводим стат. информацию по столбцу со временем между нашими событиями
advert['time_diff'].describe()
count                           99
mean     0 days 00:07:24.151515151
std      0 days 00:10:30.667485136
min                0 days 00:00:03
25%         0 days 00:00:59.500000
50%                0 days 00:02:58
75%         0 days 00:09:35.500000
max                0 days 01:04:30
Name: time_diff, dtype: object
Несмотря на то, что наблюдений у нас гораздо больше - почти в 4 раза - ситуация аналогичная - медианное значение меньше среднего, в данном случае сильно меньше, а значит у нашего распределения правостороння ассиметрия.

Если сравнивать значения по нашим рекомендациям и поисковой активности юзеров, то можно с уверенностью утверждать, что когда пользователи имеют конкретный запрос, то они быстрее переходят к просмтору контактов - оно и неудивительно. Ведь, кликая по рекомендации, юзер, возможно, даже и не задумывался о покупке данной вещи, поэтому ему требуется больше времени на то, чтобы перейти к просмотру контактов.

Кажется, что разница в 5,5 минут между среднем пройденным времени по поисковой активности и рекомендациям вполне приемлемое значение, тоже что и по медианному - разница 5 минут.

Таким образом, если судить по данной метрике - прошедшее время от просмотра объявления до просмотра контактов - то можно сказать, что наши рекомендации довольно точны и соответсвуют интересам наших юзеров, далее мы проверим еще и конверсию в целевое действие по данным событиям.

# выделяем айди юзеров, использующих реки
tips_usr = (mob_df[mob_df['event_name'] == 'tips_click']
                                .groupby('user_id', as_index=False)
                                .agg(base_event_time=('event_time', 'min')))

# выделяем айди юзеров, которые посмотрели контакты и мерджим с предыдущим датафреймом
tips_contacts_usr = (mob_df[mob_df['event_name'] == 'contacts_show']
                                .merge(tips_usr, on='user_id', how='left'))

# фильтруем, чтобы остались только те целевые события, что были после "tips_click"
tips_contacts_usr = tips_contacts_usr[tips_contacts_usr['event_time'] > tips_contacts_usr['base_event_time']]
 # считаем CR из "tips_click" в целевое действие - "contacts_show"
tips_contacts_usr['user_id'].nunique() / tips_usr['user_id'].nunique()
0.2267080745341615
# выделяем айди юзеров, которые совершали "advert_open"
advert_usr = (mob_df[mob_df['event_name'] == 'advert_open']
                                .groupby('user_id', as_index=False)
                                .agg(base_event_time=('event_time', 'min')))

# выделяем айди юзеров, которые посмотрели контакты и мерджим с предыдущим датафреймом
advert_contacts_usr = (mob_df[mob_df['event_name'] == 'contacts_show']
                                .merge(advert_usr, on='user_id', how='left'))

# фильтруем, чтобы остались только те целевые события, что были после "tips_click"
advert_contacts_usr = advert_contacts_usr[advert_contacts_usr['event_time'] > advert_contacts_usr['base_event_time']]
 # считаем CR из "advert_open" в целевое действие - "contacts_show"
advert_contacts_usr['user_id'].nunique() / advert_usr['user_id'].nunique()
0.15312916111850866
По конверсии хорошо видно, что наши юзеры гораздо чаще переходят к просмотру контактов от наших рекомендаций - 22,67%, по сравнению с тем, когда они просматривают объявления, которые нашли сами - 15,31%. А значит нашим алгоритмы внутри рекомендательной системы предлагают именно то, что интересно юзерам, однако не стоит забывать, что в среднем от клика по рекомендации до просмотра контактов проходит на 5,5 минут больше, чем просто при переходе на карточку объявления.

Данный факт можно объяснить тем, что юзерам может предлагаться товар из категории, о которой он даже не думал, но является "комплементарной" (связанной с категориями, из которых он смотрит товары), что и делает рекомендации более высококонверсионным каналом, но с большим временем между запросами контактов, ведь юзеру нужно изучить информацию о новой для него категории товаров.

Проверка гипотезы: "пользователи одинаково конвертятся в просмотр контактов из поиска и рекомендаций"
# конверсия из рекомендаций в просмотор контактов
tips_cr = tips_contacts_usr['user_id'].nunique() / len(tips_usr)
tips_cr
0.2267080745341615
# выделяем айди юзеров, использующих реки
search_usr = (mob_df[mob_df['event_name'] == 'search']
                                .groupby('user_id', as_index=False)
                                .agg(base_event_time=('event_time', 'min')))

# выделяем айди юзеров, которые посмотрели контакты и мерджим с предыдущим датафреймом
search_contacts_usr = (mob_df[mob_df['event_name'] == 'contacts_show']
                                .merge(search_usr, on='user_id', how='left'))

# фильтруем, чтобы остались только те целевые события, что были после tips_show
search_contacts_usr = search_contacts_usr[search_contacts_usr['event_time'] > search_contacts_usr['base_event_time']]
# конверсия из поиска в просмотор контактов
serach_cr = search_contacts_usr['user_id'].nunique() / len(search_usr)
serach_cr
0.20528211284513806
Как можно заметить, конверсия в просмотр контактов разная, однако без проведения стат. теста мы не можем с уверенностью утверждать, что мы имеем стат. значимое различие, так что переходим к проведению стат. теста

H0: конверсия в просмотр контактов из поиска и рекомендаций одинаковая;
H1: конверсия в просмотр контактов из поиска и рекомендаций разная;
alpha = 0.05  # критический уровень статистической значимости

cr_combined = ((tips_contacts_usr['user_id'].nunique() + search_contacts_usr['user_id'].nunique())
                / (len(tips_usr) + len(search_usr)))

difference =  tips_cr - serach_cr

z_value = difference / mth.sqrt(cr_combined * (1 - cr_combined) * (1/len(tips_usr) + 1/len(search_usr)))

distr = st.norm(0, 1)

p_value = (1 - distr.cdf(abs(z_value))) * 2

print('p-значение: ', p_value)

if p_value < alpha:
    print('Отвергаем нулевую гипотезу: между конверсиями есть значимая разница')
else:
    print(
        'Не получилось отвергнуть нулевую гипотезу, нет оснований считать конверсии разными'
    )
p-значение:  0.3864817999613277
Не получилось отвергнуть нулевую гипотезу, нет оснований считать конверсии разными
P-value > 0.05, а значит мы принимаем нулевую гипотезу - "конверсия в просмотр контактов из поиска и рекомендаций одинаковая"

Несмотря на то, что конверсия просмотров контактов из поиска и из рекомендаций разная - 20,53% и 22,67% соответственно, данные различия в данном случае не являются стат. значимыми.

Однако факт того, что конверсия в целевое действие - просмотр контактов - из поисковой активности меньше, чем из рекомендаций свидетельствует о том, что наши алгоритмы хоть и уступают в некоторых ситуациях - когда у пользователя есть конкретный запрос - но в целом, предложенные нами объявления достаточно точно попадают в интересы наших юзеров.

Шаг 5. Итоги исследования
в среднем наши юзеры совершают по 17 действий, медианное же значение - 9 действий;
для юзеров, использующих рекомендации, доля "tips_show" в общем кол-ве событий больше на 20%, 71,4% против 50,4%;
топ 5 событий юзеров, использующих рекомендации, выглядит следующим образом (название события - доля в общем кол-ве логов):
показ рекомендаций - 71,4%;
просмотр контактов - 6,9%;
клик на рекомендацию - 6,5%;
просмотр карты объявлений - 5,3%;
открытие карточки объявления - 5,28%.
а для юзеров, игнорирующих рекомендации:
показ рекомендаций - 50,4%;
просмотр фото - 16,2%;
поисковая активность - 10,4%;
открытие карточки объявления - 8,9%;
просмотр контактов - 5,9%.
доля пользователей, просмотревших контакты, больше всего в источнике – "yandex" - 24,7%, чуть меньше в "google" c 24,4%, а наименьшая доля в "other" - лишь 18,5%, если мы хотим растить долю пользователей, запрашивающих контакты, то в первую очередь стоит смотреть на масштабирование каналов "yandex" или "google", при условии, что стоимость привлечения одинаковая для всех каналов;
конверсия в "contacts_show" выше при переходе от рекомендаций "tips_click" - 22,67%, по сравнению с переходом от "advert_open" - 15,31%
наиболее популярным двухэтапным сценарием использования приложения является "tips_show" -> "contacts_show" - конверсия в целевое действие составила 17,7%;
наибольшая конверсия в целевое действие среди двухэтапных сценариев у пути от "photos_show" к "contacts_show" - 27,5%;
среди трехэтапных сессий наибольшая конверсия в целевое действие у сценария "map" -> "tips_show" -> "contacts_show" - 18,9%;
чем меньше шагов до целевого события, тем выше конверсия в его совершение.
Рекомендации
Как мне кажется, проведение пользовательских интервью для каждого сегмента юзеров - тех, кто обычно приходит в приложение с конкретным запросом (сессии начинаются с "search"); тех, кто готов полистать наши рекомендации (сессии начинаются с "tips_show" или содержат много данных логов внутри сессии) и для остальных сегментов, рассмотренных в исследовании позволит спроектировать сценарии использования с учетом особенностей каждого сегмента, что в конечном итоге может приветси к сокращению средней длины пути до целевого действия. А как было сказано выше, что чем меньше шагов до целевого действия, тем выше конверсия;
сосредоточиться на развитии рекомендательной системы, поскольку юзеры, использующие реки, значительно больше времени проводят в приложении (в среднем 4,5 сессии против 2,2), а также проявляют большую вовлеченность;
поскольку просмотр контактов является важной метрикой для нашего сервиса, то крайне важно сделать эту функцию более заметной, удобной и интуитивно понятной для юзеров, здесь опять же помогут пользовательские интервью;
при масштабировании каналов рекламы смотреть прежде всего в сторону "Яндекс" и "Google" - каналов с высокой конверсией в целевое действие - просмотров контактов;
используйте опыт успешных (по конверсии) двухэтапных и трехэтапных сценариев: по данным сценарии:
"tips_show" -> "contacts_show" имеет конверсию 17,7%;
"photos_show" -> "contacts_show" самый высоко конверсионный сценарий - 27,5% - среди двухэтапных сценариев.
оптимизируйте и продвигайте эти сценарии, чтобы большее число пользователей выполнить целевое действие, аналогично с трехэтапными сценариями, "map" -> "tips_show" -> "contacts_show" имеет самый высокий коэффициент конверсии - 18,9%. На примере данного трехэтапного путя хорошо видно, насколько влияют качественные рекомендации на конверсию в целевое действие;
регулярно отслеживать основные метрики по выделенным сегментам пользователей;
провести A/B тест, при попытке масштабирования одного из высоко конверсионного сценария, чтобы не обрушить конверсию по другим сценариям.